# Dev Notebook

This notebook loads the version of trulens_eval from the enclosing repo folder. You can use this to debug or devlop trulens_eval features.

In [ ]:
# ! pip uninstall -y trulens_eval # ==0.18.2
# ! pip list | grep trulens

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Enables: Debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
from concurrent.futures import as_completed
import json
import os
from pathlib import Path
from time import sleep

import dotenv
from tqdm.auto import tqdm

from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval.feedback.provider.endpoint.base import Endpoint
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.schema import Cost
from trulens_eval.schema import FeedbackMode
from trulens_eval.schema import Record
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

In [ ]:
# Setup Tru and/or dashboard.

tru = Tru()

tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

# Keys testing

In [ ]:
# Show keys.

import os
for k in os.environ:
    if "KEY" in k:
        print(k)

# Bedrock testing


In [ ]:
from trulens_eval import Bedrock
bedrock = Bedrock(
    model_id = "amazon.titan-tg1-large",
    region_name="us-west-2"
)

In [ ]:
Endpoint.print_instrumented()

In [ ]:
bedrock_response, cost = Endpoint.track_all_costs_tally(
    thunk=lambda: bedrock.endpoint.client.invoke_model_with_response_stream(
    body=json.dumps({'inputText': "Hello there."}), modelId="amazon.titan-tg1-large")
)

In [ ]:
cost

# Huggingface testing

In [ ]:
from trulens_eval import Huggingface
Huggingface()

In [ ]:
Endpoint.print_instrumented()

# Dummy endpoint testing

In [ ]:
tp = TP()

d = Dummy(
    loading_prob=0.1,
    freeze_prob=0.0, # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.01,
    overloaded_prob=0.1,
    rpm=6000
)

# Langchain testing

In [ ]:
# Imports from langchain to build app. You may need to install langchain first
# with the following:
# ! pip install langchain>=0.0.170
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

# Initialize Huggingface-based feedback function collection class:
# bedrock = Bedrock(model_engine="Bedrock", model_id = "anthropic.claude-v2", region_name="us-west-2")

# Define a language match feedback function using HuggingFace.
f_relevance = Feedback(bedrock.relevance).on_input_output()
# By default this will check language match on the main app input and main app
# output.

full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

tru_recorder = tru.Chain(
    chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_relevance]
)

with tru_recorder as recording:
    llm_response = chain.run("What's the capital of the USA?")

display(llm_response)

# Langchain expressions testing

In [ ]:
from trulens_eval import TruChain, Tru
tru = Tru()

from langchain_core.runnables.base import RunnableSerializable
from trulens_eval.instruments import instrument
instrument.method(RunnableSerializable, "invoke")
instrument.method(RunnableSerializable, "ainvoke")

from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2_input = {"person": "obama", "language": "spanish"}

chain2.invoke(chain2_input)

tru_recorder = TruChain(
    chain2,
    app_id='Chain2'
)

with tru_recorder as recs:
    llm_response = chain2.invoke({"person": "obama", "language": "spanish"})

In [ ]:
recs.get()

# TruBasicApp testing

In [ ]:
from trulens_eval import TruBasicApp

SCRIPT_DIR = Path().cwd()
dotenv.load_dotenv(SCRIPT_DIR / "my.env")

tru = Tru(database_redact_keys=True)#database_url=os.environ.get("database_url"))

def llm_standalone(prompt):
    return prompt

f_sentiment = Feedback(bedrock.sentiment).on_output()

recorder = TruBasicApp(llm_standalone, app="default", feedbacks=[f_sentiment])